# [Day 11](https://adventofcode.com/2020/day/11): Seating System

In [1]:
import numpy as np

with open("../data/11.txt", "r") as f:
    lines = [l.strip() for l in f.readlines()]

seatingplan = np.array(list(map(list, lines)))

## Part 1

In [2]:
def reseating(seats, crowdedness, crowded):
    """Reseat according to a measure of crowdedness."""
    def reseat(occupied):
        c = crowdedness(occupied)
        return (~occupied & seats & (c == 0)) | (occupied & (c < crowded))
    return reseat

def adjacent(occupied):
    """Count occupied seats that are adjacent."""
    return np.sum([np.roll(occupied, s, axis=(0, 1)) for s in shifts], axis=0)

shifts = ((-1, -1), (-1, 0), (-1, 1),
          ( 0, -1),          ( 0, 1),
          ( 1, -1), ( 1, 0), ( 1, 1))

def fixedpoint(f, x):
    """Fixed point of a pure, type-preserving array transformation."""
    while ((fx := f(x)) != x).any():
        x = fx
    return fx

seats = np.pad(seatingplan == "L", 1)
reseat = reseating(seats, crowdedness=adjacent, crowded=4)
assert 2344 == np.sum(fixedpoint(reseat, seats))

## Part 2

In [3]:
import numba as nb
import numpy.ma as ma

UNOCCUPIED = -np.inf

def visible(seats):
    """Count occupied seats that are in direct line of sight."""
    def lineofsight(occupied):
        occ = occupied.astype(float)
        occ[seats & ~occupied] = UNOCCUPIED
        return np.sum([illum(occ) & seats for illum in illuminate], axis=0)
    return lineofsight

def illumdown(x):
    """Rays of downward illumination."""
    return shift(reluadd.accumulate(x, axis=0) > 0)

@nb.vectorize([nb.float64(nb.float64, nb.float64)], nopython=True)
def reluadd(x, y):
    return np.maximum(0, x + y)

def shift(x):
    return np.pad(x[:-1], ((1, 0), (0, 0)))

def illumup(x):
    """Rays of upward illumination."""
    return illumdown(x[::-1])[::-1]

def illumdiag(diags, illum):
    """Rays of downward diagonal illumination."""
    def illuminate(x):
        d = diags(x)
        return illum(d.data)[~d.mask].reshape(x.shape)
    return illuminate

def diagonals(shear):
    def diags(x):
        """Diagonals as columns."""
        m, n = x.shape
        i, j = np.indices((m, n))
        j = j + shear(m)[:, np.newaxis]
        diag = np.full((m, m + n - 1), UNOCCUPIED)
        mask = np.ones(diag.shape, dtype=bool)
        diag[i, j] = x
        mask[i, j] = False
        return ma.array(diag, mask=mask)
    return diags

diags = diagonals(lambda m: np.arange(m - 1, -1, -1))
antidiags = diagonals(np.arange)

illuminate = (
    illumdown,                        # S
    illumup,                          # N
    lambda x: illumdown(x.T).T,       # E
    lambda x: illumup(x.T).T,         # W
    illumdiag(diags, illumdown),      # SE
    illumdiag(diags, illumup),        # NW
    illumdiag(antidiags, illumdown),  # SW
    illumdiag(antidiags, illumup))    # NE

seats2 = seats[1:-1, 1:-1]
reseat2 = reseating(seats2, crowdedness=visible(seats2), crowded=5)
assert 2076 == np.sum(fixedpoint(reseat2, seats2))